# 02. Defect Detection - 모델 평가

이 노트북은 학습된 YOLOv11 모델의 성능을 평가하고 다양한 지표를 확인합니다.

## 평가 과정
1. 환경 설정 및 학습된 모델 로드
2. 검증 데이터셋에 대한 평가 수행
3. 성능 지표 분석 (mAP, Precision, Recall, F1-Score)
4. 결과 저장 및 시각화

## 1. 환경 설정

In [ ]:
# Ultralytics YOLO 라이브러리 설치
!pip install ultralytics -q

In [ ]:
import os
import numpy as np
from datetime import datetime
from ultralytics import YOLO
from IPython.display import Image, display

print("✅ 라이브러리 로드 완료")

## 2. Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. 작업 디렉토리 및 모델 경로 설정

**중요**: 
- `WORK_DIR`: 본인의 Google Drive 경로로 수정
- `MODEL_PATH`: 01_train.ipynb에서 학습한 모델 경로 (고정되어 있음)

In [ ]:
# Google Drive 내 작업 디렉토리 경로 설정
WORK_DIR = '/content/drive/MyDrive/2026_AI_Advanced_Study-main/3차시/01_Defect_Detection'

# 학습된 모델 경로 (고정된 경로)
MODEL_PATH = os.path.join(WORK_DIR, 'runs/defect/weights/best.pt')

# 작업 디렉토리로 이동
os.chdir(WORK_DIR)
print(f"현재 작업 디렉토리: {os.getcwd()}")

# 모델 파일 존재 확인
if os.path.exists(MODEL_PATH):
    print(f"✅ 모델 발견: {MODEL_PATH}")
else:
    print(f"❌ 모델을 찾을 수 없습니다: {MODEL_PATH}")
    print("   먼저 01_train.ipynb를 실행하여 모델을 학습하세요.")

## 4. 학습된 모델 로드

In [ ]:
# 학습된 YOLO 모델 로드
model = YOLO(MODEL_PATH)
print(f"✅ 모델 로드 완료: {MODEL_PATH}")

## 5. 모델 평가 수행

검증 데이터셋(validation set)에 대해 모델을 평가합니다.

In [ ]:
# 평가 설정
IMAGE_SIZE = 640

print("=== 평가 시작 ===")
print(f"데이터: data/config.yaml")
print(f"모델: {MODEL_PATH}")
print(f"이미지 크기: {IMAGE_SIZE}")

In [ ]:
# 검증 데이터셋에 대한 평가 실행
metrics = model.val(
    data='data/config.yaml',
    imgsz=IMAGE_SIZE,
    project='runs',
    name='defect_val',
    exist_ok=True,
    verbose=True,
    device=0
)

print("\n✅ 평가 완료!")

## 6. 성능 지표 분석

In [ ]:
# 클래스 이름 매핑
class_names = {0: "defect"}

# 성능 결과를 저장할 리스트
results_lines = []

# 전체 성능 출력
print("\n" + "="*50)
print("=== 전체 성능 (Overall Performance) ===")
print("="*50)

# mAP50 (IoU threshold 0.5에서의 평균 정밀도)
map50 = metrics.box.map50
print(f"\n📊 mAP50: {map50:.4f}")
print(f"   - IoU 임계값 0.5에서의 평균 정밀도")

# mAP50-95 (IoU threshold 0.5~0.95에서의 평균 정밀도)
map = metrics.box.map
print(f"\n📊 mAP50-95: {map:.4f}")
print(f"   - IoU 임계값 0.5~0.95에서의 평균 정밀도")

# F1 Score
f1_score = metrics.box.f1
if isinstance(f1_score, np.ndarray):
    f1_score = f1_score.mean() if len(f1_score) > 0 else 0.0
print(f"\n📊 F1 Score: {f1_score:.4f}")
print(f"   - Precision과 Recall의 조화평균")

# Precision (정밀도)
if hasattr(metrics.box, 'mp'):
    precision = metrics.box.mp
    print(f"\n📊 Precision: {precision:.4f}")
    print(f"   - 모델이 결함이라고 예측한 것 중 실제 결함의 비율")

# Recall (재현율)
if hasattr(metrics.box, 'mr'):
    recall = metrics.box.mr
    print(f"\n📊 Recall: {recall:.4f}")
    print(f"   - 실제 결함 중 모델이 정확하게 검출한 비율")

results_lines.append("=== Overall Performance ===")
results_lines.append(f"mAP50: {map50:.4f}")
results_lines.append(f"mAP50-95: {map:.4f}")
results_lines.append(f"F1 Score: {f1_score:.4f}")
if hasattr(metrics.box, 'mp'):
    results_lines.append(f"Precision: {precision:.4f}")
if hasattr(metrics.box, 'mr'):
    results_lines.append(f"Recall: {recall:.4f}")

## 7. 성능 지표 설명

### 주요 지표

- **mAP50**: IoU 임계값 0.5에서의 평균 정밀도. 값이 높을수록 모델의 검출 성능이 우수함.
- **mAP50-95**: IoU 임계값 0.5~0.95 범위에서의 평균 정밀도. 더 엄격한 평가 기준.
- **Precision**: 모델이 결함이라고 예측한 것 중 실제로 결함인 비율 (False Positive 관련).
- **Recall**: 실제 결함 중 모델이 올바르게 검출한 비율 (False Negative 관련).
- **F1 Score**: Precision과 Recall의 조화평균. 두 지표의 균형을 나타냄.

## 8. 결과 저장

In [ ]:
# 평가 결과를 텍스트 파일로 저장
results_file_path = "runs/defect_val/test_results.txt"

with open(results_file_path, 'w', encoding='utf-8') as f:
    f.write(f"YOLOv11 Defect Detection - 평가 결과\n")
    f.write(f"생성 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"모델 경로: {MODEL_PATH}\n")
    f.write(f"클래스 개수: {len(class_names)}\n\n")
    
    for line in results_lines:
        f.write(line + '\n')

print(f"\n✅ 평가 결과 저장 완료: {results_file_path}")

with open(results_file_path, 'r', encoding='utf-8') as f:
    content = f.read()
    print(content)

## 9. 평가 완료

모델 평가가 완료되었습니다. 다음 단계:
- `03_visualize.ipynb`: 실제 이미지에 대한 예측 결과 시각화

In [ ]:
print("\n" + "="*50)
print("=== 평가 완료 ===")
print("="*50)
print(f"\n평가 결과 파일: {results_file_path}")
print(f"\n다음 단계:")
print("  - 03_visualize.ipynb에서 예측 결과 시각화")